# 📊 Выгрузка справочника ценных бумаг с MOEX ISS

## 📝 Описание

Этот notebook предназначен для выгрузки данных о ценных бумагах с Московской биржи (MOEX) через API ISS и сохранения их в формат Excel.

## 🚀 Возможности

✅ Получение полного справочника ценных бумаг
✅ Фильтрация по различным критериям (рынок, движок, торговый статус и др.)
✅ Поиск по коду, названию, ISIN и другим параметрам
✅ Сохранение в Excel с возможностью включения данных о торговых площадках

## 📋 Примеры использования

1. **ВЕСЬ справочник** (все бумаги) на русском, без фильтров:
   ```bash
   python dump_iss_securities_to_excel.py --out moex_securities.xlsx
   ```

2. **Только торгуемые фьючерсы** на площадке FORTS:
   ```bash
   python dump_iss_securities_to_excel.py --engine futures --market forts --is-trading 1 --out forts_securities.xlsx
   ```

3. **Поиск по подстроке** 'MOEX' (код/название/ISIN/эмитент/госрег):
   ```bash
   python dump_iss_securities_to_excel.py --q MOEX --out search_moex.xlsx
   ```

In [1]:
# 📦 Импорт необходимых библиотек
import time
from typing import Dict, Any, List

import requests
import pandas as pd

# 🌐 Конфигурация API
BASE = "https://iss.moex.com"
PATH = "/iss/securities.json"
HEADERS = {"User-Agent": "Mozilla/5.0 (iss-securities-dumper)"}
PAGE_LIMIT = 100  # можно менять (MOEX обычно позволяет 100..1000)

print("✅ Библиотеки успешно импортированы")

✅ Библиотеки успешно импортированы


In [2]:
# 🛠️ Вспомогательные функции
def _get_json(params: Dict[str, Any]) -> Dict[str, Any]:
    """Один запрос к /iss/securities.json."""
    url = BASE + PATH
    r = requests.get(url, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()


def _table_to_df(section: dict) -> pd.DataFrame:
    """Секция формата {'columns': [...], 'data': [[...], ...]} -> DataFrame."""
    if not section or not section.get("data"):
        return pd.DataFrame()
    cols = section["columns"]
    data = section["data"]
    return pd.DataFrame(data, columns=cols)


def fetch_table_full(iss_only: str, base_params: Dict[str, Any]) -> pd.DataFrame:
    """
    Скачивает ПОЛНУЮ таблицу (постранично) из /iss/securities.json.
    iss_only: 'securities' или 'boards' (допустимые таблицы в этом эндпоинте).
    """
    frames: List[pd.DataFrame] = []
    start = 0
    while True:
        params = dict(base_params)
        params["iss.only"] = iss_only
        params["start"] = start
        params["limit"] = base_params.get("limit", PAGE_LIMIT)
        js = _get_json(params)
        df = _table_to_df(js.get(iss_only, {}))
        if df.empty:
            break
        frames.append(df)
        # если вернулось меньше лимита — следующей страницы нет
        if len(df) < params["limit"]:
            break
        start += params["limit"]
        time.sleep(0.2)  # чуть-чуть подождём, чтобы не спамить
    if frames:
        out = pd.concat(frames, ignore_index=True)
        # на всякий случай уберём дубли
        out = out.drop_duplicates()
        return out
    return pd.DataFrame()

In [3]:
# 📥 Основная функция для получения данных
def fetch_securities_data(q=None, lang="ru", engine=None, market=None, 
                         is_trading=None, group_by=None, group_by_filter=None, 
                         limit=PAGE_LIMIT, include_boards=False,
                         filter_stock_futures=False, aggregate_by_shortname=False):
    """
    Получает данные о ценных бумагах с MOEX ISS API
    
    Параметры:
    - filter_stock_futures: если True, фильтрует только фьючерсы на акции (emitent_title содержит 'публичное')
    - aggregate_by_shortname: если True, группирует по shortname и агрегирует данные
    """
    base_params: Dict[str, Any] = {"lang": lang, "limit": limit}

    # эти параметры добавляем только если заданы
    if q:
        base_params["q"] = q
    if engine:
        base_params["engine"] = engine
    if market:
        base_params["market"] = market
    if is_trading is not None:
        base_params["is_trading"] = is_trading
    if group_by:
        base_params["group_by"] = group_by
    if group_by_filter:
        base_params["group_by_filter"] = group_by_filter

    # 1) securities
    print("📥 Загружаю таблицу 'securities' ...")
    df_securities = fetch_table_full("securities", base_params)
    print(f"📊 securities: {len(df_securities)} строк")

    # Фильтрация фьючерсов на акции
    if filter_stock_futures and not df_securities.empty:
        if 'emitent_title' in df_securities.columns:
            # Фильтруем по emitent_title содержащему 'публичное' (в любом регистре)
            mask = df_securities['emitent_title'].str.contains('публичное', case=False, na=False)
            df_securities = df_securities[mask]
            print(f"🔍 После фильтрации фьючерсов на акции: {len(df_securities)} строк")
        else:
            print("⚠️ Колонка 'emitent_title' не найдена, фильтрация пропущена")

    # Обработка shortname и группировка
    if aggregate_by_shortname and not df_securities.empty:
        if 'shortname' in df_securities.columns:
            # Разделяем shortname по '-' и берем только первый элемент
            df_securities['shortname_formatted'] = df_securities['shortname'].str.split('-').str[0]
            
            # Группируем по отформатированному shortname
            # Для остальных колонок используем first (можно изменить на last)
            agg_dict = {}
            for col in df_securities.columns:
                if col not in ['shortname', 'shortname_formatted']:
                    agg_dict[col] = 'first'  # или 'last' в зависимости от предпочтений
            
            df_securities = df_securities.groupby('shortname_formatted').agg(agg_dict).reset_index()
            print(f"📊 После группировки по shortname: {len(df_securities)} строк")
        else:
            print("⚠️ Колонка 'shortname' не найдена, группировка пропущена")

    # 2) boards (опционально)
    df_boards = pd.DataFrame()
    if include_boards:
        print("📥 Загружаю таблицу 'boards' ...")
        df_boards = fetch_table_full("boards", base_params)
        print(f"📊 boards: {len(df_boards)} строк")
    
    return df_securities, df_boards

In [4]:
# 🧪 Пример использования
print("🚀 Пример получения данных о ценных бумагах")
print("📈 Получение фьючерсов на акции с группировкой по shortname")

# Загрузка данных с новой функциональностью
df_securities, df_boards = fetch_securities_data(
    engine="futures", 
    market="forts", 
    is_trading=1, 
    limit=100,
    filter_stock_futures=True,  # Фильтруем только фьючерсы на акции
    aggregate_by_shortname=True  # Группируем по shortname
)

print(f"✅ Получено {len(df_securities)} записей в securities")
if not df_securities.empty:
    print("📋 Первые 5 строк данных:")
    display(df_securities.head())
    
    # Показываем уникальные значения отформатированного shortname
    if 'shortname_formatted' in df_securities.columns:
        print(f"\n📊 Уникальные базовые активы: {df_securities['shortname_formatted'].nunique()}")
        print("🔍 Примеры базовых активов:")
        print(df_securities['shortname_formatted'].value_counts().head(10))

🚀 Пример получения данных о ценных бумагах
📈 Получение фьючерсов на акции с группировкой по shortname
📥 Загружаю таблицу 'securities' ...
📊 securities: 396 строк
🔍 После фильтрации фьючерсов на акции: 131 строк
📊 После группировки по shortname: 63 строк
✅ Получено 63 записей в securities
📋 Первые 5 строк данных:


,shortname_formatted,secid,regnumber,name,isin,is_traded,emitent_id,emitent_title,emitent_inn,emitent_okpo,type,group,primary_boardid,marketprice_boardid
0,AFKS,AKH6,None,Фьючерсный контракт AFKS-3.26,None,1,777.0,"Публичное акционерное общество ""Акционерная фи...",7703104630,27987276,futures,futures_forts,RFUD,None
1,AFLT,AFH6,None,Фьючерсный контракт AFLT-3.26,None,1,1242.0,"Публичное акционерное общество ""Аэрофлот – рос...",7712040126,29063984,futures,futures_forts,RFUD,None
2,ALRS,ALH6,None,Фьючерсный контракт ALRS-3.26,None,1,1670.0,"Акционерная компания ""АЛРОСА"" (публичное акцио...",1433000147,23308410,futures,futures_forts,RFUD,None
3,ASTR,ASH6,None,Фьючерсный контракт ASTR-3.26,None,1,14935.0,Публичное акционерное общество Группа Астра,7726476459,None,futures,futures_forts,RFUD,None
4,BANE,BNH6,None,Фьючерсный контракт BANE-3.26,None,1,2997.0,"Публичное акционерное общество ""Акционерная не...",0274051582,00135645,futures,futures_forts,RFUD,None



📊 Уникальные базовые активы: 63
🔍 Примеры базовых активов:
shortname_formatted
AFKS      1
AFLT      1
ALRS      1
ASTR      1
BANE      1
BELUGA    1
BSPB      1
CBOM      1
CHMF      1
FEES      1
Name: count, dtype: int64


In [ ]:
# 🔍 Дополнительный пример: сравнение с группировкой и без
print("\n" + "="*60)
print("🔍 Сравнение результатов с группировкой и без")

# Без группировки
print("\n📊 БЕЗ группировки:")
df_no_group, _ = fetch_securities_data(
    engine="futures", 
    market="forts", 
    is_trading=1, 
    limit=100,
    filter_stock_futures=True,
    aggregate_by_shortname=False
)
print(f"Количество записей: {len(df_no_group)}")

# С группировкой
print("\n📊 С группировкой:")
df_with_group, _ = fetch_securities_data(
    engine="futures", 
    market="forts", 
    is_trading=1, 
    limit=100,
    filter_stock_futures=True,
    aggregate_by_shortname=True
)
print(f"Количество записей: {len(df_with_group)}")

if not df_with_group.empty and 'shortname_formatted' in df_with_group.columns:
    print(f"Уникальных базовых активов: {df_with_group['shortname_formatted'].nunique()}")
    print("\n🔍 Топ-10 базовых активов:")
    print(df_with_group['shortname_formatted'].value_counts().head(10))


In [ ]:
# 💾 Сохранение в Excel
def save_to_excel(df_securities, df_boards, filename="moex_securities.xlsx"):
    """
    Сохраняет данные в Excel-файл
    """
    with pd.ExcelWriter(filename, engine="xlsxwriter") as xw:
        df_securities.to_excel(xw, index=False, sheet_name="securities")
        if not df_boards.empty:
            df_boards.to_excel(xw, index=False, sheet_name="boards")
    
    print(f"✅ Данные успешно сохранены в файл: {filename}")

# 📁 Пример сохранения
save_to_excel(df_securities, df_boards, "moex_securities_example.xlsx")

In [ ]:
save_to_excel(df_securities, df_boards, "moex_securities_example.xlsx")

## 📚 Документация по параметрам

| Параметр | Описание | Тип |
|---------|----------|-----|
| `q` | Поиск по части кода/названия/ISIN/эмитента/рег.номеру | str |
| `lang` | Язык (ru или en) | str |
| `engine` | Фильтр по движку (stock, futures, currency, ...) | str |
| `market` | Фильтр по рынку (shares, forts, selt, index, ...) | str |
| `is_trading` | 1 — только торгуемые, 0 — неторгуемые | int (0/1) |
| `group_by` | Группировка (group|type) | str |
| `group_by_filter` | Фильтр по группе (зависит от group_by) | str |
| `limit` | Размер страницы (по умолчанию 500) | int |
| `include_boards` | Также выгрузить таблицу boards | bool |

## 🎯 Примеры значений параметров

**Движки (engine):**
- `stock` - Фондовый рынок
- `futures` - Срочный рынок
- `currency` - Валютный рынок
- `commodity` - Товарный рынок
- `interventions` - Рынок интервенций

**Рынки (market):**
- `shares` - Рынок акций
- `bonds` - Рынок облигаций
- `forts` - Фьючерсы и опционы
- `selt` - Поставочные валютные контракты
- `otc` - ОТС-система
- `ccp` - CCP система клиринга